<a href="https://colab.research.google.com/github/Matrixx1229/EcoBite/blob/main/Twitter_Sentiment_Analysis_with_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle

In [2]:

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [3]:
#API to fetch dataset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 85% 69.0M/80.9M [00:00<00:00, 188MB/s]
100% 80.9M/80.9M [00:00<00:00, 171MB/s]


In [4]:
#extracting the compressed dataset

from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The Dataset is extracted')

The Dataset is extracted


In [5]:
#importing the dependencies
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#printing stopwords in English
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Processing

In [8]:
# loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [9]:
# checkin the number of rows and columns
twitter_data.shape

(1599999, 6)

In [10]:
# printing the first 5 rows of dataframe
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
# nameing the columns and reading the dataset again

column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_names, encoding = 'ISO-8859-1')

In [12]:
# checking
twitter_data.shape

(1600000, 6)

In [13]:
#printing first 5 rows
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
# counting the number of the missing values in the dataset
twitter_data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [15]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

Convert the target "4" to "1"

In [16]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [17]:
# checking the distribution of target column
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative Tweet
1 --> Positive Tweet

**STEMMING**

Stemming is the process of reducing a woord to its Root word

In [18]:
port_stem = PorterStemmer()

In [19]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z].',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [20]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)  #50 mins for execution

KeyboardInterrupt: 

In [ ]:
twitter_data.head()


,target,ids,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",witchfoot ttp witpic om l www ummer ou houlda ...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,pset hat e pdate acebook ext ight ry esult sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,enichan ive ime manag ave est ut f ound
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,hole odi eel tchi nd ike ts n ire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",ationwideclass ot ehav ad ere ee ou ver


In [ ]:
print(twitter_data['stemmed_content'])

0          witchfoot ttp witpic om l www ummer ou houlda ...
1          pset hat e pdate acebook ext ight ry esult sch...
2                    enichan ive ime manag ave est ut f ound
3                          hole odi eel tchi nd ike ts n ire
4                    ationwideclass ot ehav ad ere ee ou ver
                                 ...                        
1599995                              oke p chool est eel ver
1599996    thewdb om eri ool ear ld alt nterview ttp lip mta
1599997                        ou eadi ojo akeov ask e etail
1599998        happi h irthday oo f lll ime tupac maru hakur
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['text'])

0          @switchfoot http://twitpic.com/2y1zl - Awww, t...
1          is upset that he can't update his Facebook by ...
2          @Kenichan I dived many times for the ball. Man...
3            my whole body feels itchy and like its on fire 
4          @nationwideclass no, it's not behaving at all....
                                 ...                        
1599995    Just woke up. Having no school is the best fee...
1599996    TheWDB.com - Very cool to hear old Walt interv...
1599997    Are you ready for your MoJo Makeover? Ask me f...
1599998    Happy 38th Birthday to my boo of alll time!!! ...
1599999    happy #charitytuesday @theNSPCC @SparksCharity...
Name: text, Length: 1600000, dtype: object


In [ ]:
# Seperating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [ ]:
print(X)

['witchfoot ttp witpic om l www ummer ou houlda ot avid arr f hird ay'
 'pset hat e pdate acebook ext ight ry esult school oday lso blah!'
 'enichan ive ime manag ave est ut f ound' ...
 'ou eadi ojo akeov ask e etail'
 'happi h irthday oo f lll ime tupac maru hakur'
 'happi charitytuesday thenspcc sparkschar speakinguph']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data to training data and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,stratify=Y, random_state=3)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [ ]:
print(X_train)

['rm ust ell f omput hair hat o ean n' 'hrishillfan ou ot edia ext lucki'
 'uck avin atch ur e ick nd nt elp er' ...
 'elosm ooop id gain sorri xist iss .'
 'rowzl ccord g achelmann l tart ain omorrow lock n orn let ope e ight ja omm uch'
 'lcarlton ol dea ate uy ore loth much impler ption']


In [ ]:
print(X_test)

['ate hen og ark ometh hen oth appen cari time ry leep.'
 'illi ay e ither ut ill upport uz ike ou uy' 'ordalynn ah retti ess p'
 ... 'use pare sb lash rive irtual c ts ot b f emori'
 'istatreen ealou iss each'
 'akura urosaki rob ope ou ave ood ne lt oh nd hat ind f ake ove ake.']


In [ ]:
# converting the textual data to numerical data

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

  (0, 103998)	0.36701825671966176
  (0, 165413)	0.2020585341995797
  (0, 161710)	0.5323611629659892
  (0, 310351)	0.4257520809002004
  (0, 121345)	0.27889348727081176
  (0, 428750)	0.24831762447847086
  (0, 368882)	0.46941824197353155
  (1, 259212)	0.4378312897991696
  (1, 146199)	0.26639664865848883
  (1, 110295)	0.4157726409809736
  (1, 328416)	0.16617588218060417
  (1, 330206)	0.14799342772103952
  (1, 184790)	0.7176024106710505
  (2, 132512)	0.28668019668602124
  (2, 123277)	0.3313063676738081
  (2, 290578)	0.4285956568727481
  (2, 279687)	0.16812510937585873
  (2, 192346)	0.29440947742049334
  (2, 424980)	0.3230130296172863
  (2, 53774)	0.2748872202908284
  (2, 62946)	0.49878174508585893
  (2, 412592)	0.28412971556379996
  (3, 426984)	0.2766537124850604
  (3, 404439)	0.3595125175796819
  (3, 334764)	0.17376702359489518
  :	:
  (1279997, 388704)	0.2812273752707224
  (1279998, 4889)	0.4655706053931387
  (1279998, 373048)	0.4524333174825584
  (1279998, 228294)	0.31477978314528643
  (

In [ ]:
print(X_test)

  (0, 402923)	0.3170211532344073
  (0, 377261)	0.24583378810095394
  (0, 328921)	0.27652158977867775
  (0, 308540)	0.27782765413538674
  (0, 300543)	0.30144403898147004
  (0, 247453)	0.23926970525065913
  (0, 172123)	0.414261917343714
  (0, 80056)	0.3728673940509753
  (0, 54028)	0.22148732196809717
  (0, 43473)	0.305679937621904
  (0, 37413)	0.29009059578134727
  (1, 432809)	0.39964138147288736
  (1, 432620)	0.30111121416197445
  (1, 429890)	0.194565118796195
  (1, 424646)	0.4253806034097234
  (1, 330206)	0.1728477617002601
  (1, 223685)	0.39717418531464754
  (1, 204206)	0.4372854001027863
  (1, 203766)	0.24575078930312033
  (1, 200836)	0.23216499049952655
  (1, 64850)	0.19410904952112848
  (2, 360992)	0.3303492074633356
  (2, 321909)	0.7960427062635886
  (2, 138397)	0.3552510067944014
  (2, 12335)	0.3619145383988494
  :	:
  (319997, 379983)	0.384833847751719
  (319997, 368011)	0.2860027306523731
  (319997, 338300)	0.40183915084163935
  (319997, 328416)	0.1456263710436811
  (319997, 24

Training the machine learning model

In [ ]:
# logistic regression

model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy score

In [ ]:
# Accuracy score on the training data

X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.8014578125


In [ ]:
# Accuracy score on the test data

X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.768646875


Model accuracy = 76.8%

In [ ]:
#saving the trained model

import pickle

In [ ]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

Using the saved model for future predictions

In [ ]:
# loading the save model
loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [ ]:
X_new = X_test [145]
print(Y_test[145])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The Tweet is Negative')
else:
  print('The Tweet is Positive')



0
[0]
The Tweet is Negative
